# News Cloud

Облако тегов на основании наиболее частых слов, встречающихся на лентах трех крупнейших российских информагентств:

```
--------------------------
Media             | TIC
--------------------------
1. TASS           | 13308
2. RIA            | 13079
3. Interfax       | 4448
```

## Пространство имен

In [23]:
# Импорты

import psycopg2 as psql              # Базы данных
import requests                      # HTTP-запросы
from bs4 import BeautifulSoup as Bs  # Парсинг
import re                            # Регулярные выражения
import pandas as pd                  # Таблицы и фильтры
import time                          # Тайминги
from datetime import datetime
from collections import Counter      # Счетчик

## Регистрационные данные
from auth import auth

## Обработка естественного языка
import pymorphy2
import nltk
# nltk.download('punkt')
# nltk.download('wordnet')
# nltk.download('stopwords')
from nltk.corpus import stopwords
# from nltk.corpus import wordnet

## Часто используемый код

In [3]:
class Parser:
    """Основной парсер"""

    def __init__(self, url):
        self.url = url

    def get_parser_refs(self):
        """Возвращает список ссылок на страницы новостей"""

        source = requests.get(self.url).text

        # Объект для разбора
        if "ria" in self.url:
            soup = Bs(source, 'lxml').find('span', class_='share')
        elif "tass" in self.url:
            soup = Bs(source, 'lxml').find('main', class_='container')
        elif "interfax" in self.url:
            soup = Bs(source, 'lxml').find('main')
        else:
            pass

        refs = []
        raw_result = soup.find_all_next('a', href=True)

        # Список всех ссылок по видам информагентств
        if "ria" in self.url:
            for ref in raw_result:
                if all(elm in ref.get('href') for elm in ['html', 'http']):
                    refs.append(ref.get('href'))

        elif "tass" in self.url:
            for ref in raw_result:
                if re.search('[0-9]', ref.get('href')) and all(elm not in ref.get('href') for elm in ['http', 'press']):
                    refs.append(ref.get('href'))

        elif "interfax" in self.url:
            for ref in raw_result:
                if re.search('[0-9]', ref.get('href')) and all(
                        elm not in ref.get('href') for elm in [
                            'http', 'photo', 'html', 'story', 'pressreleases', 'asp', 'aeroflot'
                        ]):
                    refs.append(ref.get('href'))

        else:
            pass

        return refs

    def get_parser_body(self, ref):
        """Возвращает первые 4k символов из текста новости по ссылке"""

        result = str()

        if "ria" in self.url:
            source = requests.get(ref).text
            soup = Bs(source, 'lxml')
            raw_result = soup.find_all('div', class_='article__text')

            for i in raw_result:
                result = result + i.text + ' '

        elif "interfax" in self.url:
            source = requests.get(f'{self.url}{ref}')
            source.encoding = 'cp1251'
            source = source.text
            soup = Bs(source, 'lxml').find('div', class_='mainblock')
            raw_result = soup.find_all_next('p')

            for i in raw_result:
                result = result + i.text + ' '

        elif "tass" in self.url:
            source = requests.get(f'{self.url}{ref}').text
            soup = Bs(source, 'lxml')
            raw_result = soup.find_all('div', class_='text-content')
            result = result + raw_result[0].text

        else:
            pass

        return result[:4000]

In [4]:
def morph_normalizer(word):
    """Приведение слова к инфинитиву (лемматизация)"""
    
    morph_analyzer = pymorphy2.MorphAnalyzer(lang='ru')
    normal_form = morph_analyzer.parse(word)[0].normal_form
    return normal_form

## Токенизация текстов и создание датасета

In [5]:
def create_dataset(urls):
    """Создает датасет из наиболее часто встречающихся слов
    в новостях информационных агентств
    """
    
    normal_form = []
       
    # Список стоп-слов - для очистки текста от мусора
    stop_words = set(stopwords.words("russian"))

    for url in urls:
        parser = Parser(url)
        refs = parser.get_parser_refs()
        for ref in refs:
            body = parser.get_parser_body(ref)
            text = re.sub(r"[^\w]", " ", body.lower())
            words = nltk.word_tokenize(text)
            
            # Применение стоп-слов, удаление их из текста
            without_stop_words = [
                word for word in words if not word in stop_words
            ]
            
            # Приведение слова к инфинитиву (лемматизация)
            for word in without_stop_words:
                if len(word) >=4:
                    normal_form.append(morph_normalizer(word))

            time.sleep(2)  # Задержка между запросами в сек.
    
    # Подсчет количества слов в списке
    counter = Counter(normal_form)
    df = pd.DataFrame(counter.items(), columns=['word', 'count'])
    
    return df

In [6]:
data = create_dataset(['https://www.ria.ru',
                       'https://www.interfax.ru',
                       'https://tass.ru']
                     )

In [17]:
data = data.sort_values(by=['count'], ascending=False).head(30)
data['day'] = datetime.now().strftime("%Y-%m-%d")
data

,word,count,day
155,год,276,2020-12-26
55,который,184,2020-12-26
0,москва,184,2020-12-26
103,декабрь,182,2020-12-26
44,человек,171,2020-12-26
68,россия,171,2020-12-26
3,коронавирус,119,2020-12-26
412,новый,115,2020-12-26
12,сообщить,111,2020-12-26
1,новость,99,2020-12-26


## Работа с базой данных

In [21]:
def insert_into_db(data):
    """Вставляет переданные в функцию данные в БД"""

    # Авторизация в базе данных
    conn = psql.connect(
        dbname=auth.psql_db,
        user=auth.psql_user,
        password=auth.psql_passwd,
        host='localhost'
    )

    cursor = conn.cursor()
    print('Создано подключение к БД:', auth.psql_db)
    print('Выполнение запроса...')    

    for index, row in data.iterrows():
        values = (
            row['word'],
            row['count'],
            f"{row['day']}"
        )

        cursor.execute(
            """
            INSERT INTO 
            newscloud (word, count, day) 
            VALUES (%s, %s, %s)
            """, values
        )

    conn.commit()

    cursor.close()
    print('Подключение к БД завершено')

    conn.close()
    print('Соединение с БД закрыто')

In [24]:
# Сохранение полученных данных в БД

insert_into_db(data)
%time

Создано подключение к БД: etl
Выполнение запроса...
Подключение к БД завершено
Соединение с БД закрыто
CPU times: user 4 µs, sys: 1e+03 ns, total: 5 µs
Wall time: 17.9 µs
